# Aula 04 - Introdução ao Spacy

Professor: Luiz Frias 
- Email: l.frias@poli.ufrj.br
- Linkedin: in/luizfrias/
- Twitter: @lfdefrias

**Objetivo:** entendimento das funcionalidades básicas do pacote de NLP Spacy

### 101

Melhores momentos de https://spacy.io/usage/spacy-101 utilizando o modelo de Português https://spacy.io/models/pt

In [1]:
import spacy
from spacy import displacy
import pandas as pd

nlp = spacy.load("pt_core_news_md")

In [2]:
nlp

In [5]:
doc = nlp("Vou pegar um copo d'água e já volto. Aguardem.")
doc, type(doc)

(Vou pegar um copo d'água e já volto. Aguardem., spacy.tokens.doc.Doc)

In [6]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Vou AUX aux
pegar VERB ROOT
um DET det
copo NOUN obj
d'água PROPN nmod
e CCONJ cc
já ADV advmod
volto VERB conj
. PUNCT punct
Aguardem VERB ROOT
. PUNCT punct


In [7]:
list(doc.sents)

[Vou pegar um copo d'água e já volto., Aguardem.]

In [11]:
list(doc.sents)[1], type(list(doc.sents)[1])

(Aguardem., spacy.tokens.span.Span)

In [9]:
[token for token in list(doc.sents)[1]]

[Aguardem, .]

In [12]:
type(doc[0]), doc[0].text

(spacy.tokens.token.Token, 'Vou')

Você pode conferir todos os atributos do token aqui: https://spacy.io/api/token#attributes

O Spacy conta com um modelo de POS/DEP e disponibiliza os resultados via atributos

In [13]:
_data = []

doc = nlp("A aula de NLP está acontecendo remotamente há 2 semanas.")

for token in doc:
    _data.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])

pd.DataFrame(_data, columns=['Texto', 'Lemma', 'POS', 'TAG', 'DEP', 'Shape', 'is_alpha', 'is_stop'])

,Texto,Lemma,POS,TAG,DEP,Shape,is_alpha,is_stop
0,A,A,DET,DET__Definite=Def|Gender=Fem|Number=Sing|PronT...,det,X,True,False
1,aula,aula,NOUN,NOUN__Gender=Fem|Number=Sing,nsubj,xxxx,True,False
2,de,de,ADP,ADP,case,xx,True,True
3,NLP,NLP,PROPN,PROPN__Gender=Masc|Number=Sing,nmod,XXX,True,False
4,está,estar,AUX,AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|...,aux,xxxx,True,True
5,acontecendo,acontecer,VERB,VERB__VerbForm=Ger,ROOT,xxxx,True,False
6,remotamente,remotamente,ADV,ADV,advmod,xxxx,True,False
7,há,haver,VERB,VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres...,advcl,xx,True,False
8,2,2,NUM,NUM__NumType=Card,nummod,d,False,False
9,semanas,semana,NOUN,NOUN__Gender=Fem|Number=Plur,obj,xxxx,True,False


In [14]:
spacy.explain("ADP")

'adposition'

In [15]:
displacy.render(doc, style="dep", jupyter=True, options={'distance': 100})

In [23]:
pd.DataFrame([(token.text, token.head.text) for token in doc], columns=['Texto', 'Head'])

,Texto,Head
0,A,aula
1,aula,acontecendo
2,de,NLP
3,NLP,aula
4,está,acontecendo
5,acontecendo,acontecendo
6,remotamente,acontecendo
7,há,acontecendo
8,2,semanas
9,semanas,há


O Spacy conta também com um modelo de NER treinado nas seguintes classes LOC, MISC, ORG, PER

In [16]:
doc = nlp("As olimpíadas de 2016 ocorreram no Rio de Janeiro, Brasil.")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Rio de Janeiro 35 49 LOC
Brasil 51 57 LOC


In [26]:
displacy.render(doc, style="ent", jupyter=True, options={'distance': 100})

Os modelos MD e LG possuem a capacidade de representar as palavras como vetores (word embeddings)

In [17]:
doc.vector.shape

(300,)

In [20]:
doc1 = nlp("As Olimpíadas de 2016 ocorreram no Rio de Janeiro, Brasil.")
doc2 = nlp("A Copa do Mundo de 2022 será no Catar")
doc3 = nlp("Este é só um documento de teste")

In [21]:
doc1.similarity(doc2), doc1.similarity(doc3), doc2.similarity(doc3)

(0.5272187365392528, 0.07238869183351607, 0.3272669485052952)

In [22]:
nlp('Copa do Mundo').similarity(doc1), nlp('Copa do Mundo').similarity(doc2), nlp('Copa do Mundo').similarity(doc3)

(0.4226053845017395, 0.7553999269834669, 0.20113580019954544)

In [23]:
from scipy import spatial

In [24]:
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

print("janeiro vs fevereiro", 
      cosine_similarity(
          nlp.vocab['janeiro'].vector, nlp.vocab['fevereiro'].vector)
     )

janeiro vs fevereiro 0.9674742817878723


In [26]:
nlp('janeiro').similarity(nlp('fevereiro'))

0.9674744116712797

In [39]:
homem = nlp.vocab['homem'].vector
mulher = nlp.vocab['mulher'].vector
rainha = nlp.vocab['rainha'].vector
rei = nlp.vocab['rei'].vector
calculated_rei = homem - mulher + rainha
print("Similaridade entre Rei calculado e Rei", cosine_similarity(calculated_rei, rei))

Similaridade entre Rei calculado e Rei 0.7316041588783264


Por falar em vocabulário... Podemos pensar em `nlp.vocab.strings` como um mapa bidirecional entre string e hash. Ele é particularmente útil para economia de espaço e processamento, visto que as propriedades do token que são independentes de contexto podem ser apenas recuperadas utilizando-se esse mapeamento.

In [42]:
nlp.vocab.strings['homem']

12392230024392888241

In [43]:
nlp.vocab.strings[12392230024392888241]

'homem'

Finalizamos o 101 com os exemplos de https://spacy.io/usage/spacy-101#lightning-tour

### Pipeline

A arquitetura do Spacy funciona de forma a criar um pipeline de execução

tokenizer => tagger => parser => ner => personalizada

A primeira etapa não pode ser desabilitada, mas as outras podem. E eventualmente novas etapas podem ser adicionadas.

In [27]:
nlp = spacy.load('pt_core_news_md')

In [28]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [29]:
nlp = spacy.load('pt_core_news_md', disable = ['ner', 'tagger', 'parser'])

In [30]:
nlp.pipe_names

[]

### Tokenization

Doc vs Span vs Token:

- Doc é a principal estrutura de dados, é composto por tokens
- Span representa uma fatia do Doc
- Token representa uma unidade fundamental do Doc/Span

In [31]:
doc = nlp("Vou pegar um copo d'água e já volto. Aguardem.")
[token.text for token in doc]

['Vou',
 'pegar',
 'um',
 'copo',
 "d'água",
 'e',
 'já',
 'volto',
 '.',
 'Aguardem',
 '.']

In [32]:
type(doc), doc.text

(spacy.tokens.doc.Doc, "Vou pegar um copo d'água e já volto. Aguardem.")

In [126]:
type(doc[0]), doc[0]

(spacy.tokens.token.Token, Vou)

In [127]:
span = doc[0:1]
type(span), span.text

(spacy.tokens.span.Span, 'Vou')

In [33]:
token = doc[1]
[m for m in dir(token) if m.startswith('is_')]

['is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper']

https://github.com/explosion/spaCy/blob/master/spacy/lang/lex_attrs.py

Tokens podem ter métodos adicionados 

In [34]:
from spacy.tokens import Doc, Token, Span

time_getter = lambda token: token.text.lower() in ("vasco", "flamengo", "fluminense", "botafogo")
Token.set_extension("is_time", getter=time_getter, force=True)

In [37]:
doc = nlp("Vou torcer pro Vasco ser campeão.")
token = doc[3]
token.text, token._.is_time

('Vasco', True)

Observe apenas que, caso estes métodos dependam de uma etapa do pipeline que foi desabilitado previamente, eles podemo não funcionar corretamente.

In [38]:
substantivo_getter = lambda token: token.pos_ == 'NOUN'
Token.set_extension("is_substantivo", getter=substantivo_getter, force=True)

doc = nlp("Vou torcer pro meu time ser campeão.")
token = doc[4]
token.text, token._.is_substantivo

('time', False)

In [39]:
token.pos_

''

In [40]:
nlp = spacy.load('pt_core_news_md')

In [42]:
doc = nlp("Vou torcer pro meu time ser campeão.")
token = doc[4]
token.text, token._.is_substantivo

('time', True)

In [43]:
doc = nlp('Oi 😊. Está tudo 🆗?')
[token.text for token in doc]

['Oi', '😊', '.', 'Está', 'tudo', '🆗', '?']

### Sentence Boundary Detection

In [44]:
doc = nlp("Vou pegar um copo d'água e já volto. Aguardem.")

In [45]:
list(doc.sents)

[Vou pegar um copo d'água e já volto., Aguardem.]

### POS Tagging

In [46]:
doc = nlp("Vou pegar um copo d'água e já volto. Aguardem.")
[(token.text, token.pos_, token.tag_) for token in doc]

[('Vou', 'AUX', 'AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin'),
 ('pegar', 'VERB', 'VERB__VerbForm=Inf'),
 ('um', 'DET', 'DET__Definite=Ind|Gender=Masc|Number=Sing|PronType=Art'),
 ('copo', 'NOUN', 'NOUN__Gender=Masc|Number=Sing'),
 ("d'água", 'PROPN', 'PROPN__Gender=Masc|Number=Sing'),
 ('e', 'CCONJ', 'CCONJ'),
 ('já', 'ADV', 'ADV'),
 ('volto',
  'VERB',
  'VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin'),
 ('.', 'PUNCT', 'PUNCT'),
 ('Aguardem',
  'VERB',
  'VERB__Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin'),
 ('.', 'PUNCT', 'PUNCT')]

In [47]:
spacy.explain('CCONJ')

'coordinating conjunction'

https://universaldependencies.org/treebanks/pt_bosque/index.html

In [48]:
doc = nlp('Oi 😊. Está tudo 🆗?')
[(token.text, token.pos_, token.tag_) for token in doc]

[('Oi', 'ADV', 'ADV'),
 ('😊', 'INTJ', 'INTJ'),
 ('.', 'PUNCT', 'PUNCT'),
 ('Está', 'AUX', 'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin'),
 ('tudo', 'PRON', 'PRON__Gender=Masc|Number=Sing|PronType=Ind'),
 ('🆗', 'ADV', 'ADV'),
 ('?', 'PUNCT', 'PUNCT')]

In [50]:
doc = nlp('Oi 😊. Está tudo ok?')
[(token.text, token.pos_, token.tag_) for token in doc]

[('Oi', 'ADV', 'ADV'),
 ('😊', 'INTJ', 'INTJ'),
 ('.', 'PUNCT', 'PUNCT'),
 ('Está', 'AUX', 'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin'),
 ('tudo', 'PRON', 'PRON__Gender=Masc|Number=Sing|PronType=Ind'),
 ('ok', 'INTJ', 'INTJ'),
 ('?', 'PUNCT', 'PUNCT')]

In [51]:
doc = nlp('Oi 😊. Está tudo bem?')
[(token.text, token.pos_, token.tag_) for token in doc]

[('Oi', 'ADV', 'ADV'),
 ('😊', 'INTJ', 'INTJ'),
 ('.', 'PUNCT', 'PUNCT'),
 ('Está', 'AUX', 'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin'),
 ('tudo', 'PRON', 'PRON__Gender=Masc|Number=Sing|PronType=Ind'),
 ('bem', 'ADV', 'ADV'),
 ('?', 'PUNCT', 'PUNCT')]

In [102]:
spacy.explain('INTJ')

'interjection'

### Dependency Parsing

In [52]:
doc = nlp("Vou dar uma volta de carro autônomo.")

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

In [53]:
displacy.render(doc, style="dep", jupyter=True, options={'distance': 100})

In [54]:
pd.DataFrame([
    (token.text, token.pos_, token.head.text, token.head.pos_, list(token.head.children)) for token in doc
], columns=['Token', 'POS', 'Head', 'Head POS', 'Head Children'])

,Token,POS,Head,Head POS,Head Children
0,Vou,AUX,dar,VERB,"[Vou, volta, autônomo, .]"
1,dar,VERB,dar,VERB,"[Vou, volta, autônomo, .]"
2,uma,DET,volta,NOUN,"[uma, carro]"
3,volta,NOUN,dar,VERB,"[Vou, volta, autônomo, .]"
4,de,ADP,carro,NOUN,[de]
5,carro,NOUN,volta,NOUN,"[uma, carro]"
6,autônomo,ADJ,dar,VERB,"[Vou, volta, autônomo, .]"
7,.,PUNCT,dar,VERB,"[Vou, volta, autônomo, .]"


In [55]:
doc[3], doc[3].pos, doc[3].pos_, doc[3].dep, doc[3].dep_

(volta, 92, 'NOUN', 434, 'obj')

In [57]:
from spacy.symbols import obj, NOUN

In [58]:
doc[3].pos == NOUN, doc[3].pos_ == 'NOUN', doc[3].dep == obj, doc[3].dep_ == 'obj'

(True, True, True, True)

In [59]:
doc[1].n_lefts, list(doc[1].lefts), doc[1].n_rights, list(doc[1].rights)

(1, [Vou], 3, [volta, autônomo, .])

In [182]:
list(doc[1].ancestors), list(doc[1].children)

[Vou, volta, autônomo, .]

In [183]:
list(doc[3].ancestors), list(doc[3].children)

([dar], [uma, carro])

### Lemmatization

In [191]:
doc = nlp('Demonstrando lemmatizer do Spacy para os alunos')
[(token, token.lemma, token.lemma_) for token in doc]

[(Demonstrando, 6761547791254441684, 'Demonstrando'),
 (lemmatizer, 11400258643245852636, 'lemmatizer'),
 (do, 2158845516055552166, 'do'),
 (Spacy, 6947170135922310690, 'Spacy'),
 (para, 9374817464001885516, 'parir'),
 (os, 1489474827855109852, 'o'),
 (alunos, 3754566140822040580, 'aluno')]

In [63]:
from nltk.stem import RSLPStemmer
stem_getter = lambda token: RSLPStemmer().stem(token.text)
Token.set_extension("stem", getter=stem_getter, force=True)

In [64]:
doc = nlp('Demonstrando lemmatizer do Spacy para os alunos')
[(token, token.lemma, token.lemma_, token._.stem) for token in doc]

[(Demonstrando, 6761547791254441684, 'Demonstrando', 'demonstr'),
 (lemmatizer, 11400258643245852636, 'lemmatizer', 'lemmatiz'),
 (do, 2158845516055552166, 'do', 'do'),
 (Spacy, 6947170135922310690, 'Spacy', 'spacy'),
 (para, 9374817464001885516, 'parir', 'par'),
 (os, 1489474827855109852, 'o', 'os'),
 (alunos, 3754566140822040580, 'aluno', 'alun')]

In [188]:
nlp.vocab.strings[6761547791254441684]

'Demonstrando'